In [ ]:
import os

import cv2
import matplotlib.pyplot as plt
import numpy as np
import skimage
import skimage.io
from matplotlib.colors import ListedColormap
from skimage import exposure

np.random.seed(0)
import os

import cupy as cp
import h5py
import psutil
import skimage

p = psutil.Process(os.getpid())

In [ ]:
import os

import cupy as cp
import h5py
import matplotlib.pyplot as plt
import numpy as np
import skimage.io
from skimage.io import imread, imsave
from skimage.transform import rescale
from tqdm.notebook import tqdm

In [ ]:
def create_MIP(im_proj):
    im_proj = skimage.exposure.rescale_intensity(im_proj, out_range=(0, 1))
    im_proj = cp.asarray(im_proj)
    vmin_green, vmax_green = cp.percentile(im_proj[0, :], q=(0.01, 99.9)).get()
    vmin_magenta, vmax_magenta = cp.percentile(im_proj[1, :], q=(0.01, 99.9)).get()
    im_proj = im_proj.get()

    clipped_green = exposure.rescale_intensity(
        im_proj[0, :], in_range=(vmin_green, vmax_green), out_range=np.float32
    )

    clipped_magenta = exposure.rescale_intensity(
        im_proj[1, :], in_range=(vmin_magenta, vmax_magenta), out_range=np.float32
    )
    green_map = [[0, i / 255, 0] for i in range(256)]
    green_matplotlib_map = ListedColormap(green_map, "Green")
    magenta_map = [[i / 255, 0, i / 255] for i in range(256)]
    magenta_matplotlib_map = ListedColormap(magenta_map, "Magenta")
    channel1 = green_matplotlib_map(clipped_green)
    channel2 = magenta_matplotlib_map(clipped_magenta)

    assembled = cp.stack((cp.asarray(channel1), cp.asarray(channel2)), axis=3)
    newim = cp.max((assembled), axis=3).get()
    return newim


def moving_average(a, n=3):
    ret = np.cumsum(a, dtype=float)
    ret[n:] = ret[n:] - ret[:-n]
    return ret[n - 1 :] / n

In [ ]:
def color_image(image, mask, color, mask_type):
    # combine
    colormask_organoid = np.zeros(image.shape, image.dtype)
    colormask_organoid[:, :] = color
    colormask_organoid = cv2.bitwise_and(
        colormask_organoid,
        colormask_organoid,
        mask=(mask == mask_type).astype(np.uint8),
    )
    image = cv2.addWeighted(colormask_organoid, 0.2, image, 1, 0, image)
    return image

In [ ]:
mempool = cp.get_default_memory_pool()
pinned_mempool = cp.get_default_pinned_memory_pool()

In [ ]:
positions = np.arange(1, 17)
# positions=[2,4]

for position in tqdm(positions):
    # save
    mask_dir = f"/cluster/project/treutlein/DATA/imaging/viventis/20210503_201032_6_lines_mosaic_HB4_D4_processed/Position_{position}_Settings_1_Processed/lumen_masks_v19_05_2023/"
    output_folder = (
        f"/cluster/project/treutlein/DATA/imaging/viventis/20210503_201032_6_lines_mosaic_HB4_D4_processed/Position_{position}_Settings_1_Processed/"
        + "intermediate_movie_files/biggest_slice_v13_07_23/"
    )

    plt.style.use("dark_background")

    import os

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    stacks = np.linspace(1, 249, 125, dtype=int)
    largest_slice_list = []
    largest_mask_image = []
    for stack in tqdm(stacks):
        mempool.free_all_blocks()
        pinned_mempool.free_all_blocks()

        try:
            combined_masks = cp.array(
                imread(
                    mask_dir + f"{stack:04}" + "_lumen_organoid_mask_processed.tif"
                ).copy()
            )
            organoid_mask = combined_masks >= 2
            non_zero_elements = cp.count_nonzero(organoid_mask, axis=(1, 2))
            # print(non_zero_elements.get())
            # end=cp.where(non_zero_elements>=20000)[0][-1]
            # start=cp.where(non_zero_elements>=20000)[0][0]
            # middle_slice=int((end*0.6+0.3*start))
            percentile_diff = cp.abs(
                cp.percentile(non_zero_elements, 90) - non_zero_elements
            )

            middle_slice = int(cp.where(percentile_diff == percentile_diff.min())[0][0])

            largest_slice_list.append(middle_slice)
            largest_slice_old = middle_slice
        except:
            print("failed percentile")
            largest_slice_list.append(largest_slice_old)

    largest_slice_list = np.array(largest_slice_list)
    if position == 2:
        largest_slice_list[50:] = largest_slice_list[49]
    # elif position==4:
    #    largest_slice_list[43:49]=largest_slice_list[42]
    # largest_slice_list_mv=moving_average(largest_slice_list, n=20).astype(int)
    window = 20
    largest_slice_list_mv = np.lib.stride_tricks.sliding_window_view(
        np.pad(
            largest_slice_list, (0, window - 1), constant_values=largest_slice_list[-1]
        ),
        window,
    ).mean(axis=-1)
    largest_slice_list_mv = np.round(largest_slice_list_mv).astype(int)
    all_stack = []
    all_image_false_colored = []
    all_stack_sep_channel = []

    stack_file = h5py.File(
        f"/cluster/project/treutlein/DATA/imaging/viventis/20210503_201032_6_lines_mosaic_HB4_D4_processed/Position_{position}_Settings_1_Processed/denoised_registered_processed.h5",
        "r",
    )
    for t, largest_slice in tqdm(zip(stacks, largest_slice_list_mv)):
        try:
            combined_masks = imread(
                mask_dir + f"{t:04}" + "_lumen_organoid_mask_processed.tif"
            ).copy()
            organoid_mask = (combined_masks >= 2).astype(bool)
            lumen_mask = skimage.morphology.remove_small_objects(
                (combined_masks == 3), min_size=20000 / (2 * 1.388 * 1.388)
            ).astype(bool)
            combined_masks = np.ones(combined_masks.shape)
            combined_masks[organoid_mask] = 2
            combined_masks[lumen_mask] = 3

        except:
            print("fail ")
            combined_masks = np.zeros(combined_masks.shape)
        stack_cherry = (
            stack_file[f"t{t:05}"]["s00"]["0"]["cells"][largest_slice]
            .astype(np.float32)
            .copy()
        )
        stack_gfp = (
            stack_file[f"t{t:05}"]["s01"]["0"]["cells"][largest_slice]
            .astype(np.float32)
            .copy()
        )
        stack_combined = np.stack([stack_cherry, stack_gfp])
        image = stack_gfp + stack_cherry
        image *= 255.0 / np.percentile(image.copy(), q=99.9)
        image = image.clip(0, 255)
        image = np.stack((image.astype(np.uint8),) * 3, axis=-1)
        mask = skimage.transform.resize(
            combined_masks[largest_slice], image.shape[:2], order=0, preserve_range=True
        ).astype("uint8")
        image_false_colored = color_image(image, mask, (58, 252, 249), 3)
        image_false_colored = color_image(image_false_colored, mask, (232, 229, 46), 2)
        # plt.imshow(image_false_colored)
        # plt.show()
        all_image_false_colored.append(image_false_colored)
        all_stack.append(stack_combined)
        stack_sep_channel = []
        stack_combined_ch = stack_combined.sum(0)
        for masked_ind in range(1, 4):
            stack_sep_channel.append((mask == masked_ind) * stack_combined_ch)
        stack_sep_channel = np.array(stack_sep_channel)
        all_stack_sep_channel.append(stack_sep_channel)
    movie_folder = f"/cluster/project/treutlein/DATA/imaging/viventis/20210503_201032_6_lines_mosaic_HB4_D4_processed/lumen_movies_TIFF_v13_07_23/"
    # movie_folder=f'/cluster/project/treutlein/DATA/imaging/viventis/20210503_201032_6_lines_mosaic_HB4_D4_processed/lumen_movies_TIFF_v12_06_23/'
    if not os.path.exists(movie_folder):
        os.makedirs(movie_folder)
    all_stack = np.array(all_stack)
    all_image_false_colored = np.array(all_image_false_colored)
    all_stack_sep_channel = np.array(all_stack_sep_channel)
    """
    imsave(movie_folder+f'90_percentile_slice_positions_{position}_movie.tiff',
       all_stack,imagej=True, 
       metadata={'axes': 'TCYX'}, compression ='zlib')

    
    """

    imsave(
        movie_folder + f"90_percentile_slice_positions_{position}.tiff",
        all_stack_sep_channel,
        imagej=True,
        resolution=(1.0 / (0.347), 1.0 / (0.347)),
        metadata={"unit": "um", "axes": "TCYX"},
        compression="zlib",
    )

    imsave(
        movie_folder + f"90_percentile_slice_positions_{position}_colored.tiff",
        np.moveaxis(all_image_false_colored, -1, 1),
        imagej=True,
        metadata={"axes": "TCYX"},
        compression="zlib",
    )

    import imageio

    writer = imageio.get_writer(
        movie_folder + f"largest_slice_positions_{position}_colored.mp4", fps=10
    )
    for im in tqdm(all_image_false_colored):
        writer.append_data(im)
    writer.close()

In [ ]:
def padding(array, xx, yy):
    """
    :param array: numpy array
    :param xx: desired height
    :param yy: desirex width
    :return: padded array
    """

    h = array.shape[1]
    w = array.shape[2]

    a = (xx - h) // 2
    aa = xx - a - h

    b = (yy - w) // 2
    bb = yy - b - w

    return np.pad(array, pad_width=((0, 0), (a, aa), (b, bb), (0, 0)), mode="constant")

In [ ]:
# movie_folder=f'/cluster/project/treutlein/DATA/imaging/viventis/20210503_201032_6_lines_mosaic_HB4_D4_processed/biggest_slice_v13_07_23/'
movie_folder = f"/cluster/project/treutlein/DATA/imaging/viventis/20210503_201032_6_lines_mosaic_HB4_D4_processed/lumen_movies_TIFF_v13_07_23/"
positions = np.arange(1, 17)
all_images = []
for position in tqdm(positions):
    false_col_image = imread(
        movie_folder + f"90_percentile_slice_positions_{position}.tiff"
    ).copy()

    if position == 2:
        false_col_image[50:] = 0
    # elif position==4:
    #    false_col_image[43:49]=0
    false_col_image_downscaled = []
    # Downscale
    for img in tqdm(false_col_image):
        false_col_image_downscaled.append(rescale(img, [0.125, 0.125, 1]))

    false_col_image_downscaled = np.array(false_col_image_downscaled)
    # Pad to size
    false_col_image_downscaled = padding(false_col_image_downscaled, 300, 300)

    all_images.append(false_col_image_downscaled)

all_images = np.array(all_images)
stack_vert = []
for i in range(0, len(all_images), 4):
    stack_vert.append(np.dstack(all_images[i : (i + 4)]))
# stack_vert.append(np.dstack(all_images[i:(i+4)]))
# stack_vert.append(np.dstack(all_images[i:(i+4)]))
# stack_vert.append(np.dstack(all_images[i:(i+4)]))

stack_vert = np.array(stack_vert)
stack_vert = np.hstack(stack_vert)

imsave(
    movie_folder + f"grid_90_percentile_slice_positions_multimo.tiff",
    np.moveaxis(stack_vert, -1, 1),
    imagej=True,
    resolution=(1.0 / (8 * 0.347), 1.0 / (8 * 0.347)),
    metadata={"unit": "um", "axes": "TCYX"},
    compression="zlib",
)